#  Link PyPower

In [1]:
from collections import OrderedDict

import pandas as pd

import ams
import andes

print(ams.__version__)
print(andes.__version__)

0.5+86.gbf9899c.dirty
1.8.5


In [2]:
# NOTE: get a pypower case
from ams.solver.ipp import load_ppc

ppc = load_ppc(ams.get_case('pypower/case14.py'))

ppc.keys()

dict_keys(['version', 'baseMVA', 'bus', 'gen', 'branch', 'gencost'])

In [3]:
ppc['gen'][:, 11:21]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [4]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
                no_output=True,
                setup=True,)

In [5]:
sp = ams.load(ams.get_case('ieee14/ieee14.xlsx'),
            setup=True)

In [6]:
dct = sp.as_dict()

In [7]:
dct.keys()

odict_keys(['Bus', 'PQ', 'PV', 'Slack', 'Shunt', 'Line', 'Area', 'GCost'])

In [8]:
bus_df = sp.Bus.as_df()
pq_df = sp.PQ.as_df()
pv_df = sp.PV.as_df()
slack_df = sp.Slack.as_df()
gen_df = pd.concat([pv_df, slack_df], ignore_index=True)
line_df = sp.Line.as_df()

In [9]:
line_df.columns

Index(['idx', 'u', 'name', 'bus1', 'bus2', 'Sn', 'fn', 'Vn1', 'Vn2', 'r', 'x',
       'b', 'g', 'b1', 'g1', 'b2', 'g2', 'trans', 'tap', 'phi', 'rate_a',
       'rate_b', 'rate_c', 'owner', 'xcoord', 'ycoord', 'amin', 'amax'],
      dtype='object')

In [10]:
from ams.solver.ipp import to_ppc

spc, key_dict, bus_type, ppc_bus, ppc_gen, ppc_line = to_ppc(sp)

In [11]:
key_dict['Bus'].keys()

odict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

In [12]:
bus_idx = pd.DataFrame()
bus_idx['bus_ssp'] = key_dict['Bus'].keys()
bus_idx['bus_i'] = key_dict['Bus'].values()

In [13]:
key_dict['Bus']

OrderedDict([(1, 1),
             (2, 2),
             (3, 3),
             (4, 4),
             (5, 5),
             (6, 6),
             (7, 7),
             (8, 8),
             (9, 9),
             (10, 10),
             (11, 11),
             (12, 12),
             (13, 13),
             (14, 14)])

In [14]:
ppc_line

,fbus,tbus,r,x,b,rateA,rateB,rateC,ratio,angle,status,angmin,angmax
uid,,,,,,,,,,,,,
0,1,2,0.01938,0.05917,0.0528,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0
1,1,5,0.05403,0.22304,0.0492,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0
2,2,3,0.04699,0.19797,0.0438,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0
3,2,4,0.05811,0.17632,0.0340,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0
4,2,5,0.05695,0.17388,0.0346,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0
5,3,4,0.06701,0.17103,0.0128,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0
6,4,5,0.01335,0.04211,0.0000,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0
7,6,11,0.09498,0.19890,0.0000,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0
8,6,12,0.12291,0.25581,0.0000,NaN,NaN,NaN,1.00000,NaN,NaN,-360.0,360.0


In [15]:
spc['gen']

array([[2, 40.0, 15.0, 15.0, -40.0, 1.03, nan, nan, 50.0, 10.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 99900.0, 99900.0, 99900.0, 99900.0, 0.0],
       [3, 40.0, 15.0, 15.0, -10.0, 1.01, nan, nan, 50.0, 10.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 99900.0, 99900.0, 99900.0, 99900.0, 0.0],
       [6, 30.0, 10.0, 10.0, -6.0, 1.03, nan, nan, 50.0, 10.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 99900.0, 99900.0, 99900.0, 99900.0, 0.0],
       [8, 35.0, 10.0, 10.0, -6.0, 1.03, nan, nan, 50.0, 10.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 99900.0, 99900.0, 99900.0, 99900.0, 0.0],
       [1, 81.44200000000001, 1.9619999999999997, 100.0, -50.0, 1.03,
        nan, nan, 200.0, 50.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 99900.0,
        99900.0, 99900.0, 99900.0, 0.0]], dtype=object)

In [16]:
ppc['bus'][:, [4, 5]]

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0., 19.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])